# Основы парсинга и работа с API

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>


In [71]:
import requests

In [72]:
KEYWORDS = ['python', 'парсинг','браузеры']

In [73]:
from bs4 import BeautifulSoup

In [74]:
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')

In [75]:
posts = soup.find_all('article', class_='post')
posts

[<article class="post post_preview" lang="ru">
 <header class="post__meta">
 <a class="post__user-info user-info" href="https://habr.com/ru/users/Finnix/" title="Автор публикации">
 <img class="user-info__image-pic user-info__image-pic_small" height="24" src="//habrastorage.org/getpro/habr/avatars/fe7/d1d/8e1/fe7d1d8e145eefbb527b5edee2a90aea.jpg" width="24"/>
 <span class="user-info__nickname user-info__nickname_small">Finnix</span>
 </a>
 <span class="post__time">сегодня в 05:50</span>
 </header>
 <h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/company/southbridge/blog/525320/">Этапы внедрения CI/CD</a>
 </h2>
 <ul class="post__hubs inline-list">
 <li class="inline-list__item inline-list__item_hub">
 <a class="inline-list__item-link hub-link" href="https://habr.com/ru/company/southbridge/" onclick="if (typeof ga === 'function') { ga('send', 'event', 'hub', 'feed page', 'Блог компании Southbridge'); }" rel="nofollow" title="Вы не подписаны на этот хаб">Б

In [76]:
from datetime import datetime, timedelta

In [77]:
def get_today():
    date_format = '%d.%m.%Y'
    today = datetime.now()
    return today.strftime(date_format)
            
def get_tomorrow():
    date_format = '%d.%m.%Y'
    today = datetime.now()
    tomorrow = today + timedelta(days=1)
    return tomorrow.strftime(date_format)

In [78]:
import pandas as pd
df = pd.DataFrame(columns = ['дата', 'заголовок', 'ссылка'])
index = 0

In [79]:
for post in posts:
    post_id = post.parent.attrs.get('id')
    if not post_id:
        continue
    post_id = int(post_id.split('_')[-1])
    hubs = post.find_all('a', class_='hub-link')
    for hub in hubs:
            hub_lower = hub.text.lower()
            if any([hub_lower in desired for desired in KEYWORDS]):
                title_element = post.find('a', class_='post__title_link')
                df.loc[index, 'заголовок'] = title_element.text
                df.loc[index, 'ссылка'] = title_element.attrs.get('href')
                br = post.find('span', class_='post__time')
                if ('вчера' in br.text):
                    df.loc[index, 'дата'] = get_tomorrow()
                elif ('сегодня' in br.text):
                    df.loc[index, 'дата'] = get_today()
                else:
                    df.loc[index, 'дата'] = br.text
                index = index + 1
                break

In [80]:
df

,дата,заголовок,ссылка
0,29.10.2020,Реализуем и сравниваем оптимизаторы моделей в ...,https://habr.com/ru/company/skillfactory/blog/...
1,29.10.2020,Opera станет китайской: компания из Поднебесно...,https://habr.com/ru/company/selectel/blog/525306/
2,29.10.2020,AutoVIML: Автоматизированное машинное обучение,https://habr.com/ru/company/otus/blog/525292/
